In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models

import sys
sys.path.insert(1, '../')
from msn.model import get_model
from configs.config import get_config

In [16]:
config = get_config()
config

callback_config:
  checkpoint_filepath: wandb/model_{epoch}
  early_patience: 6
  rlrp_factor: 0.2
  rlrp_patience: 3
  save_best_only: true
  use_earlystopping: true
  use_reduce_lr_on_plateau: false
  viz_num_images: 100
dataset_config:
  batch_size: 8
  num_crops:
  - 2
  - 5
  shuffle_buffer: 100
  size_crops:
  - 224
  - 96
  use_options: true
model_config:
  attention_probs_dropout_prob: 0.0
  backbone: ViT
  decoder_hidden_size: 512
  decoder_intermediate_size: 2048
  decoder_num_attention_heads: 16
  decoder_num_hidden_layers: 8
  dropout_rate: 0.5
  hidden_act: gelu
  hidden_dropout_prob: 0.0
  hidden_size: 768
  initializer_range: 0.02
  intermediate_size: 3072
  is_encoder_decoder: false
  layer_norm_eps: 1.0e-12
  mask_ratio: 0.75
  model_img_channels: 3
  model_img_height: 224
  model_img_width: 224
  norm_pix_loss: false
  num_attention_heads: 12
  num_channels: 3
  num_hidden_layers: 12
  patch_size: 16
  post_gap_dropout: false
  qkv_bias: true
seed: 0
train_config:
  e

In [17]:
tf.keras.backend.clear_session()
model = get_model(config)

In [18]:
model

In [24]:
pixel_values = tf.random.normal((8, 224, 224, 3))
print(model(pixel_values=pixel_values))

(<tf.Tensor: shape=(8, 50, 768), dtype=float32, numpy=
array([[[-1.3337277 , -0.36170658, -0.24928866, ...,  0.566708  ,
          0.30504584,  0.18290684],
        [-1.4670463 , -0.47795087, -0.8266787 , ...,  1.3091518 ,
         -0.9643258 , -0.3065158 ],
        [-0.28448457, -0.6741753 , -1.3182832 , ...,  0.55855024,
         -0.90546083, -0.44181138],
        ...,
        [ 0.28519922,  0.83283454,  0.3355648 , ...,  0.12044197,
          0.3204322 ,  0.14033014],
        [-0.30329543, -0.03886554, -1.0101507 , ...,  1.166146  ,
         -0.65142083, -0.7088371 ],
        [-0.3351615 ,  0.55325973, -0.3631666 , ...,  1.0405645 ,
         -1.3863213 , -0.44821632]],

       [[-1.4187552 , -0.23415688, -0.44320905, ...,  0.39510465,
          0.3662171 ,  0.03615766],
        [-0.6061038 ,  0.3353727 , -0.8856694 , ...,  0.6923131 ,
         -0.14381096, -0.22352023],
        [ 0.2560666 ,  0.7859119 ,  0.25260773, ...,  0.30878803,
          0.07023957, -0.18060163],
        ...,

In [22]:
model.summary()

Model: "tf_vi_tmae_main_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embeddings (ViTMAEEmbedding  multiple                 742656    
 s)                                                              
                                                                 
 encoder (TFViTMAEEncoder)   multiple                  85054464  
                                                                 
 layernorm (LayerNormalizati  multiple                 1536      
 on)                                                             
                                                                 
Total params: 85,798,656
Trainable params: 85,647,360
Non-trainable params: 151,296
_________________________________________________________________
